In [1]:
import numpy as np
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import gc
from shutil import copyfile
from shutil import move
import re
import random

data_dir = 'D:/Downloads/amazon/'

# ['slash_burn', 'blooming', 'primary', 'conventional_mine', 'water', 'cultivation', 'artisinal_mine', 'habitation', 'bare_ground', 'blow_down', 'agriculture', 'road', 'selective_logging']
target_label = 'agriculture'

preprocess_train_dir = data_dir + 'prep/' + target_label + '/train-jpg/'
preprocess_validation_dir = data_dir + 'prep/' + target_label + '/validation-jpg/'

validation_fraction = 0.2  # fraction of original training set to be used for validation set

df = pd.read_csv(data_dir + 'train.csv')

In [6]:
# copy images to the positive/ and negative/ folder.  
# this is a preprocessing step for using Keras ImageDataGenerator.flow_from_directory.

def preprocessImageFileForIndex(index, target):
    source_filename = df.iloc[index]['image_name'] + '.jpg'
    #print(source_filename)
    #print(df.iloc[index]['tags'])

    data_src_dir = data_dir + 'train-jpg/'

    words = df.iloc[index]['tags'].split()
    if target in words:
        copyfile(data_src_dir + source_filename, preprocess_train_dir + 'positive/'+ source_filename)
    else:
        copyfile(data_src_dir + source_filename, preprocess_train_dir + 'negative/'+ source_filename)

In [8]:
def makedir(path):
    if not os.path.exists(path):
        os.makedirs(path)

makedir(preprocess_train_dir)
makedir(preprocess_train_dir + 'positive/')
makedir(preprocess_train_dir + 'negative/')
makedir(preprocess_validation_dir)
makedir(preprocess_validation_dir + 'positive/')
makedir(preprocess_validation_dir + 'negative/')

In [9]:
for index, row in df.iterrows():
    preprocessImageFileForIndex(index, target_label)

In [10]:
# Move randomized samples to validation-jpg folder
def randomTrainValidationSplit(validation_fraction, train_dir, validation_dir):
    list_files = os.listdir(train_dir)
    random.shuffle(list_files) # In-place shuffle
    validation_size = int(round(len(list_files) * validation_fraction, 0))
    print('validation_size:' , validation_size)
    validation_files = list_files[0:validation_size]
    for curr_file in validation_files:
        src = train_dir + curr_file
        dest = validation_dir + curr_file
        #print ('src: ' + src)
        #print ('dest: ' + dest)
        move(src, dest)

In [11]:
randomTrainValidationSplit(validation_fraction, preprocess_train_dir + 'positive/', preprocess_validation_dir + 'positive/')
randomTrainValidationSplit(validation_fraction, preprocess_train_dir + 'negative/', preprocess_validation_dir + 'negative/')

('validation_size:', 2468)
('validation_size:', 5628)


In [ ]:
print('preprocessing complete for ' + target_label)